I'm fairly new to this, so this is primarly for my own learning. I read through the other kernels and learned a lot, and wanted to give it a go as well.
Mostly inspired by this great kernel: https://www.kaggle.com/aakashnain/eda-nytaxi/notebook

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/train.csv')
df.head()

In [ ]:
print("Columns: {}".format(df.columns))
print("Shape: {}".format(df.shape))

In [ ]:
df.info()

No null values, which is awesome. Now lets cast the [pickup/dropoff]_datetime to a proper date-object and create seperate columns for the hour, day, week and month

In [ ]:
def create_date_columns(col, prefix, df=df):
    weekday_names = {0: "Monday", 1: "Tuesday", 2: "Wednesday", 3: "Thursday", 4: "Friday", 5: "Saturday", 6: "Sunday"}
    df[col] = df[col].apply(pd.to_datetime)
    df[prefix + "hour"] = df[col].dt.hour
    df[prefix + "day"] = df[col].dt.day
    df[prefix + "weekday"] = df[col].dt.dayofweek.map(weekday_names)
    df[prefix + "weekday_index"] = df[col].dt.weekday
    df[prefix + "month"] = df[col].dt.month
    
    return df

In [ ]:
df = create_date_columns("pickup_datetime", "pickup_")
df = create_date_columns("dropoff_datetime", "dropoff_")
df["trip_duration_min"] = df["trip_duration"] // 60
df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.countplot(x="passenger_count", data=df, hue="vendor_id", ax=ax)
plt.xlabel("Number of Passengers")

It looks like when the number of passengers is greater than 4, it is almost exclusively vendor 2. That is kind of interesting.

In [ ]:
passengers = df.set_index("pickup_datetime").sort_index()
passengers["passenger_count"].resample("D").max().plot()
passengers["passenger_count"].resample("D").min().plot()
passengers["passenger_count"].resample("D").mean().plot()

In [ ]:
plt.scatter(x=df["passenger_count"], y=df["trip_duration_min"])
plt.xlabel("Number of Passengers")
plt.ylabel("Total Duration in Minutes")

Clearly we have some outliers here. 

In [ ]:
plt.hist(df["trip_duration_min"], range=[0, 50], bins=50)
plt.xlabel("Duration in Minutes")

Right-skewed distribution. Also the mjority is between 5 - 15 minutes long

Alright - Let's have a look at when most trips where made.

In [ ]:
weekday_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
fig, ax = plt.subplots(figsize=(20, 8), ncols=2, nrows=2)
sns.countplot(x="pickup_hour", data=df, ax=ax[0][0])
sns.countplot(x="pickup_day", data=df, ax=ax[0][1])
sns.countplot(x="pickup_weekday", data=df, ax=ax[1][0], order=weekday_order)
sns.countplot(x="pickup_month", data=df, ax=ax[1][1])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 8))
sns.countplot(x="pickup_weekday", hue="pickup_hour", data=df, ax=ax)
plt.legend(loc=(1.03,0))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.countplot(x="pickup_hour", hue="passenger_count", data=df, ax=ax)
plt.legend(loc=(1.03,0))

fig, ax = plt.subplots(figsize=(20, 5))
sns.countplot(x="pickup_weekday", hue="passenger_count", data=df, ax=ax, order=weekday_order)
plt.legend(loc=(1.03,0))

fig, ax = plt.subplots(figsize=(20, 5))
sns.countplot(x="pickup_day", hue="passenger_count", data=df, ax=ax)
plt.legend(loc=(1.03,0))

fig, ax = plt.subplots(figsize=(20, 5))
sns.countplot(x="pickup_month", hue="passenger_count", data=df, ax=ax)
plt.legend(loc=(1.03,0))

Everything seems normal here.

I will continue on this notebook later. So far it's been a very good learning-experience. 